# Computational Theory Assessment

## Problem 1

In [3]:
import numpy as np

def parity(x, y, z):
    """
    Parity function: x ⊕ y ⊕ z
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of XOR operation
    """
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    return np.uint32(x ^ y ^ z)

In [ ]:
def ch(x, y, z):
    """
    Choice function: (x ∧ y) ⊕ (¬x ∧ z)
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of choice operation
    """
    # Convert inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    
    # For each bit: if x bit is 1, choose y bit; if x bit is 0, choose z bit
    return np.uint32((x & y) ^ (~x & z))

### Choice Function Explanation

The **Choice (Ch)** function acts as a bitwise multiplexer. It examines each bit position in `x`:
- If the bit in `x` is 1, it selects the corresponding bit from `y`
- If the bit in `x` is 0, it selects the corresponding bit from `z`

This function is crucial in SHA-256's compression function, providing conditional bit selection based on the first input.

**Mathematical representation:** `Ch(x,y,z) = (x ∧ y) ⊕ (¬x ∧ z)`

In [8]:
def maj(x, y, z):
    """
    Majority function: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    For each bit position, returns the majority bit (the bit that appears most frequently).
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of majority operation
    """

    # Convert inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Return the majority bit for each position
    # If at least 2 of 3 bits are 1, result bit is 1
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

### Majority Function Explanation

The **Majority (Maj)** function implements democratic bit selection. For each bit position, it returns 1 if at least two of the three input bits are 1, otherwise it returns 0.

This function provides resistance against certain cryptographic attacks by ensuring that no single input can completely control the output.

**Mathematical representation:** `Maj(x,y,z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)`

In [10]:
def rotr(value, positions):
    """
    Right rotate operation for 32-bit integers
    
    Args:
        value: 32-bit integer to rotate
        positions: number of positions to rotate right
        
    Returns:
        32-bit integer result of right rotation
    """
    # Ensure 32-bit unsigned integer
    value = np.uint32(value)
    positions = positions % 32  # Handle positions > 32
    
    # Right rotation: move bits right, wrap around to left
    return np.uint32((value >> positions) | (value << (32 - positions)))

In [11]:
def shr(value, positions):
    """
    Right shift operation for 32-bit integers
    
    Args:
        value: 32-bit integer to shift
        positions: number of positions to shift right
        
    Returns:
        32-bit integer result of right shift
    """
    # Ensure 32-bit unsigned integer and perform logical right shift
    value = np.uint32(value)
    return np.uint32(value >> positions)

### Helper Functions Explanation: Rotation and Shifting

These helper functions implement fundamental bit manipulation operations:

- **ROTR (Right Rotate):** Moves bits to the right, with bits that fall off the right end wrapping around to the left end
- **SHR (Right Shift):** Moves bits to the right, filling the left end with zeros

These operations are building blocks for the sigma functions and provide the bit diffusion necessary for cryptographic security.

In [12]:
def sigma0(x):
    """
    σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ SHR¹⁰(x)
    
    Lowercase sigma function used in SHA-256 message schedule.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of σ₀ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations and shift as specified in SHA-256 standard
    rotr_7 = rotr(x, 7)    # Right rotate by 7 positions
    rotr_18 = rotr(x, 18)  # Right rotate by 18 positions
    shr_3 = shr(x, 3)      # Right shift by 3 positions
    
    # XOR all three results together
    return np.uint32(rotr_7 ^ rotr_18 ^ shr_3)

In [13]:
def sigma1(x):
    """
    σ₁²⁵⁶(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    Lowercase sigma function used in SHA-256 message schedule.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of σ₁ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations and shift as specified in SHA-256 standard
    rotr_17 = rotr(x, 17)  # Right rotate by 17 positions
    rotr_19 = rotr(x, 19)  # Right rotate by 19 positions
    shr_10 = shr(x, 10)    # Right shift by 10 positions
    
    # XOR all three results together
    return np.uint32(rotr_17 ^ rotr_19 ^ shr_10)

### Message Schedule Functions (σ₀ and σ₁)

The lowercase sigma functions are used in SHA-256's **message schedule** to expand the 16-word message block into 64 words:

- **σ₀(x):** Combines right rotations by 7 and 18 positions with a right shift by 3 positions
- **σ₁(x):** Combines right rotations by 17 and 19 positions with a right shift by 10 positions

These functions ensure that each bit of the input influences multiple bits of the output, providing excellent diffusion properties essential for cryptographic security.

In [14]:
def Sigma0(x):
    """
    Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Uppercase Sigma function used in SHA-256 compression function.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of Σ₀ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations as specified in SHA-256 standard
    rotr_2 = rotr(x, 2)    # Right rotate by 2 positions
    rotr_13 = rotr(x, 13)  # Right rotate by 13 positions
    rotr_22 = rotr(x, 22)  # Right rotate by 22 positions
    
    # XOR all three results together
    return np.uint32(rotr_2 ^ rotr_13 ^ rotr_22)


In [15]:
def Sigma1(x):
    """
    Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    
    Uppercase Sigma function used in SHA-256 compression function.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of Σ₁ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations as specified in SHA-256 standard
    rotr_6 = rotr(x, 6)    # Right rotate by 6 positions
    rotr_11 = rotr(x, 11)  # Right rotate by 11 positions
    rotr_25 = rotr(x, 25)  # Right rotate by 25 positions
    
    # XOR all three results together
    return np.uint32(rotr_6 ^ rotr_11 ^ rotr_25)


### Compression Functions (Σ₀ and Σ₁)

The uppercase Sigma functions are used in SHA-256's **compression function** during the main hashing rounds:

- **Σ₀(x):** Combines right rotations by 2, 13, and 22 positions
- **Σ₁(x):** Combines right rotations by 6, 11, and 25 positions

Unlike the message schedule functions, these use only rotations (no shifts), ensuring that no information is lost. They provide non-linear transformations that make the hash function resistant to various cryptographic attacks.

In [16]:
# Test all implemented functions with appropriate examples
print("SHA-256 Function Implementation Tests")
print("=" * 40)

# Define test values
test_x = 0x12345678
test_y = 0x9ABCDEF0
test_z = 0x87654321

print(f"Test inputs:")
print(f"  x = 0x{test_x:08X}")
print(f"  y = 0x{test_y:08X}")
print(f"  z = 0x{test_z:08X}")
print()

# Test three-input functions
print("Three-input functions:")
print(f"  Parity(x,y,z) = 0x{parity(test_x, test_y, test_z):08X}")
print(f"  Ch(x,y,z)     = 0x{ch(test_x, test_y, test_z):08X}")
print(f"  Maj(x,y,z)    = 0x{maj(test_x, test_y, test_z):08X}")
print()

# Test single-input sigma functions
print("Message schedule functions:")
print(f"  σ₀(x) = 0x{sigma0(test_x):08X}")
print(f"  σ₁(x) = 0x{sigma1(test_x):08X}")
print()

# Test single-input Sigma functions
print("Compression functions:")
print(f"  Σ₀(x) = 0x{Sigma0(test_x):08X}")
print(f"  Σ₁(x) = 0x{Sigma1(test_x):08X}")

SHA-256 Function Implementation Tests
Test inputs:
  x = 0x12345678
  y = 0x9ABCDEF0
  z = 0x87654321

Three-input functions:
  Parity(x,y,z) = 0x0FEDCBA9
  Ch(x,y,z)     = 0x97755771
  Maj(x,y,z)    = 0x92345670

Message schedule functions:
  σ₀(x) = 0xE7FCE6EE
  σ₁(x) = 0xA1F78649

Compression functions:
  Σ₀(x) = 0x66146474
  Σ₁(x) = 0x3561ABDA


## Problem 2

In [1]:
import numpy as np
import math

def primes(n):
    """
    Generate the first n prime numbers using the Sieve of Eratosthenes algorithm.
    
    Args:
        n: Number of prime numbers to generate
        
    Returns:
        List of the first n prime numbers
    """
    if n <= 0:
        return []
    
    # Estimate upper bound for the nth prime using prime number theorem
    # For n >= 6, the nth prime is less than n * ln(n * ln(n))
    if n < 6:
        upper_bound = 15  # Safe bound for first few primes
    else:
        upper_bound = int(n * math.log(n * math.log(n))) + 100
    
    # Initialize sieve array - True means potentially prime
    sieve = [True] * (upper_bound + 1)
    sieve[0] = sieve[1] = False  # 0 and 1 are not prime
    
    # Apply Sieve of Eratosthenes
    for i in range(2, int(math.sqrt(upper_bound)) + 1):
        if sieve[i]:  # i is prime
            # Mark all multiples of i as composite
            for j in range(i * i, upper_bound + 1, i):
                sieve[j] = False
    
    # Collect first n primes
    prime_list = []
    for i in range(2, upper_bound + 1):
        if sieve[i]:
            prime_list.append(i)
            if len(prime_list) == n:
                break
    
    return prime_list

In [2]:
def extract_fractional_bits(cube_root, num_bits=32):
    """
    Extract the first num_bits of the fractional part of a floating-point number.
    
    Args:
        cube_root: Floating-point number
        num_bits: Number of fractional bits to extract (default 32)
        
    Returns:
        32-bit unsigned integer representing the fractional bits
    """
    # Get the fractional part by subtracting the integer part
    fractional_part = cube_root - math.floor(cube_root)
    
    # Multiply by 2^32 to shift fractional bits into integer range
    # Then take integer part to get the first 32 fractional bits
    shifted_fraction = fractional_part * (2 ** num_bits)
    extracted_bits = int(shifted_fraction)
    
    # Ensure result fits in 32 bits
    return np.uint32(extracted_bits)

In [3]:
def calculate_sha256_constants():
    """
    Calculate the 64 round constants for SHA-256 as specified in FIPS 180-4.
    These are the first 32 bits of the fractional parts of the cube roots 
    of the first 64 prime numbers.
    
    Returns:
        List of 64 constants as 32-bit unsigned integers
    """
    # Generate first 64 prime numbers
    first_64_primes = primes(64)
    print(f"First 10 primes: {first_64_primes[:10]}")
    print(f"Last 10 primes: {first_64_primes[-10:]}")
    print()
    
    # Calculate constants
    constants = []
    
    for i, prime in enumerate(first_64_primes):
        # Calculate cube root of the prime
        cube_root = prime ** (1/3)
        
        # Extract first 32 bits of fractional part
        constant = extract_fractional_bits(cube_root)
        constants.append(constant)
        
        # Display first few and last few calculations for verification
        if i < 5 or i >= 59:
            fractional_part = cube_root - math.floor(cube_root)
            print(f"Prime {i+1:2d}: {prime:3d} -> "
                  f"∛{prime} = {cube_root:.10f} -> "
                  f"frac = {fractional_part:.10f} -> "
                  f"K[{i}] = 0x{constant:08X}") # Display in hexadecimal
    
    return constants

In [4]:
# Calculate and display the SHA-256 round constants
print("Calculating SHA-256 Round Constants")
print("=" * 50)

sha256_constants = calculate_sha256_constants()

print("\nComplete list of 64 SHA-256 round constants:")
print("=" * 50)

# Display all constants in a formatted table
for i in range(0, 64, 4):
    row = []
    for j in range(4):
        if i + j < 64:
            row.append(f"K[{i+j:2d}] = 0x{sha256_constants[i+j]:08X}")
    print("  ".join(row))

print(f"\nTotal constants generated: {len(sha256_constants)}")

Calculating SHA-256 Round Constants
First 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Last 10 primes: [257, 263, 269, 271, 277, 281, 283, 293, 307, 311]

Prime  1:   2 -> ∛2 = 1.2599210499 -> frac = 0.2599210499 -> K[0] = 0x428A2F98
Prime  2:   3 -> ∛3 = 1.4422495703 -> frac = 0.4422495703 -> K[1] = 0x71374491
Prime  3:   5 -> ∛5 = 1.7099759467 -> frac = 0.7099759467 -> K[2] = 0xB5C0FBCF
Prime  4:   7 -> ∛7 = 1.9129311828 -> frac = 0.9129311828 -> K[3] = 0xE9B5DBA5
Prime  5:  11 -> ∛11 = 2.2239800906 -> frac = 0.2239800906 -> K[4] = 0x3956C25B
Prime 60: 281 -> ∛281 = 6.5499116201 -> frac = 0.5499116201 -> K[59] = 0x8CC70208
Prime 61: 283 -> ∛283 = 6.5654144273 -> frac = 0.5654144273 -> K[60] = 0x90BEFFFA
Prime 62: 293 -> ∛293 = 6.6418521953 -> frac = 0.6418521953 -> K[61] = 0xA4506CEB
Prime 63: 307 -> ∛307 = 6.7459967117 -> frac = 0.7459967117 -> K[62] = 0xBEF9A3F7
Prime 64: 311 -> ∛311 = 6.7751689523 -> frac = 0.7751689523 -> K[63] = 0xC67178F2

Complete list of 64 SHA-256 round c

In [6]:
# Verify against the official SHA-256 constants from FIPS 180-4
official_constants = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

print("\nVerification against official FIPS 180-4 constants:")
print("=" * 55)

matches = 0
mismatches = []

for i in range(64):
    calculated = sha256_constants[i]
    official = official_constants[i]
    
    if calculated == official:
        matches += 1
        status = "✓"
    else:
        mismatches.append(i)
        status = "✗"
    
    # Show first 5, last 5, and any mismatches
    if i < 5 or i >= 59 or calculated != official:
        print(f"K[{i:2d}]: Calculated = 0x{calculated:08X}, "
              f"Official = 0x{official:08X} {status}")

print(f"\nVerification Results:")
print(f"  Matches: {matches}/64")
print(f"  Mismatches: {len(mismatches)}")

if len(mismatches) == 0:
    print("All constants match the official SHA-256 specification!")
else:
    print(f"Mismatched indices: {mismatches}")


Verification against official FIPS 180-4 constants:
K[ 0]: Calculated = 0x428A2F98, Official = 0x428A2F98 ✓
K[ 1]: Calculated = 0x71374491, Official = 0x71374491 ✓
K[ 2]: Calculated = 0xB5C0FBCF, Official = 0xB5C0FBCF ✓
K[ 3]: Calculated = 0xE9B5DBA5, Official = 0xE9B5DBA5 ✓
K[ 4]: Calculated = 0x3956C25B, Official = 0x3956C25B ✓
K[59]: Calculated = 0x8CC70208, Official = 0x8CC70208 ✓
K[60]: Calculated = 0x90BEFFFA, Official = 0x90BEFFFA ✓
K[61]: Calculated = 0xA4506CEB, Official = 0xA4506CEB ✓
K[62]: Calculated = 0xBEF9A3F7, Official = 0xBEF9A3F7 ✓
K[63]: Calculated = 0xC67178F2, Official = 0xC67178F2 ✓

Verification Results:
  Matches: 64/64
  Mismatches: 0
All constants match the official SHA-256 specification!


## Problem 3

## Problem 4

## Problem 5